# Amazon SageMaker Model Monitoring and Clarify Explainability

SageMaker contains several integrated services to monitor models for data and model quality, bias, and explainability.

In this lab, you will learn how to:
  * Capture inference requests, results, and metadata from our pipeline deployed model.
  * Schedule a Clarify default monitor to monitor for data drift on a regular schedule.
  * Schedule a Clarify model monitor to monitor model performance on a regular schedule.
  * Schedule a Clarify bias monitor to monitor predictions for bias drift on a regular schedule.
  * Schedule Clarify explainability monitor to monitor predictions for feature attribution drift on a regular schedule.

## Setup

In [ ]:
!pip install -U sagemaker==2.101.1

In [2]:
from datetime import datetime, timedelta
import pandas as pd
import time
import csv
import json
import boto3
import sagemaker

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()

sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
    DefaultModelMonitor,
    ModelQualityMonitor,
)

from sagemaker.model_monitor.dataset_format import DatasetFormat

from sagemaker.s3 import S3Downloader, S3Uploader

In [3]:
print(f"AWS region: {region}")
# A different bucket can be used, but make sure the role for this notebook has
# the s3:PutObject permissions. This is the bucket into which the data is captured.
print(f"S3 Bucket: {default_bucket}")

# Endpoint metadata.
endpoint_name = "workshop-project-prod"
endpoint_instance_count = 1
endpoint_instance_type = "ml.m5.large"
print(f"Endpoint: {endpoint_name}")

prefix = "sagemaker/xgboost-dm-model-monitoring"
s3_key = f"s3://{default_bucket}/{prefix}"
print(f"S3 key: {s3_key}")

s3_capture_upload_path = f"{s3_key}/data_capture"
s3_ground_truth_upload_path = f"{s3_key}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
s3_baseline_results_path = f"{s3_key}/baselines"
s3_report_path = f"{s3_key}/reports"

print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {s3_ground_truth_upload_path}")
print(f"Baselines path: {s3_baseline_results_path}")
print(f"Report path: {s3_report_path}")

AWS region: us-east-1
S3 Bucket: sagemaker-us-east-1-836443424694
Endpoint: workshop-project-prod
S3 key: s3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring
Capture path: s3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/data_capture
Ground truth path: s3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/ground_truth_data/2022-08-30-18-20-30
Baselines path: s3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/baselines
Report path: s3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/reports


## Configure data capture and generate synthetic traffic

Data quality monitoring automatically monitors machine learning (ML) models in production and notifies you when data quality issues arise. ML models in production have to make predictions on real-life data that is not carefully curated like most training datasets. If the statistical nature of the data that your model receives while in production drifts away from the nature of the baseline data it was trained on, the model begins to lose accuracy in its predictions. Amazon SageMaker Model Monitor uses rules to detect data drift and alerts you when it happens.

### Initialize SageMaker Predictor for real-time requests to previously deployed model endpoint

In [4]:
# Create a Predictor Python object for real-time endpoint requests. https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html
predictor = Predictor(endpoint_name=endpoint_name, serializer=CSVSerializer())

In [5]:
# SageMaker automatically created a DataCaptureConfig when your model was deployed to an endpoint 
# in a prior lab that already had data capture enabled. Below is illustrating how create a custom 
# DataCaptureConfig with data capture enabled and update an existing endpoint.
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=s3_capture_upload_path,
)

In [6]:
# Now update endpoint with data capture enabled and provide an s3_capture_upload_path.
predictor.update_data_capture_config(data_capture_config)

---------------!

Note: updating your endpoint data config can take 3-5 min. A progress bar will be displayed in the cell above and indicates completion with `---------------!` and the cell execution number. You will see your endpoint status as `Updating` under SageMaker resources > Endpoints while this is in progress and `InService` when your updated endpoint is ready for requests.

### Invoke the deployed model endpoint to generate predictions

Now send data to this endpoint to get inferences in real time. 

With data capture enabled in the previous step, the request and response payload, along with some additional metadata, is saved to the S3 location specified in `DataCaptureConfig`.

In [98]:
# Read in training set for schema and to compute feature attribution baselines.
train_df = pd.read_csv("train-headers.csv")

In [99]:
# Use test set to create a file without headers and labels to mirror data format at inference time.
test_df = pd.read_csv("test.csv")
test_df.drop(['y_no', 'y_yes'], axis=1).sample(180).to_csv("test-samples-no-header.csv", header=False)

Now send a test batch of 180 requests to the model endpoint. These inputs will be captured along with endpoint output predictions and sent to your `s3_capture_upload_path`.

In [102]:
print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

test_sample_df = pd.read_csv("test-samples-no-header.csv")

response = predictor.predict(data=test_sample_df.to_numpy())

print("Done!")

Sending test traffic to the endpoint workshop-project-prod. 
Please wait...
Done!


### View captured data

List the data capture files stored in Amazon S3. 

There should be different files from different time periods organized in S3 based on the hour in which the invocation occurred in the format: 

`s3://{destination-bucket-prefix}/{endpoint-name}/{AllTraffic or model-variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [103]:
print("Waiting 60 seconds for captures to show up", end="")

for _ in range(60):
    capture_files = sorted(S3Downloader.list(f"{s3_capture_upload_path}/{endpoint_name}"))
    if capture_files:
        break
    print(".", end="", flush=True)
    time.sleep(1)

print("\nFound Capture Files:")
print("\n ".join(capture_files[-10:]))

Waiting 60 seconds for captures to show up
Found Capture Files:
s3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/data_capture/workshop-project-prod/AllTraffic/2022/09/01/02/05-10-177-2066556b-df6c-456f-af61-271c056ee2e7.jsonl
 s3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/data_capture/workshop-project-prod/AllTraffic/2022/09/01/02/06-10-734-c7554c6e-5593-4831-86c9-09e195857583.jsonl
 s3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/data_capture/workshop-project-prod/AllTraffic/2022/09/01/02/07-11-347-99716d93-1552-47de-94d9-5a91edacdaef.jsonl
 s3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/data_capture/workshop-project-prod/AllTraffic/2022/09/01/02/08-11-979-4f7b63f3-4384-4a0b-bb76-75f907130b0a.jsonl
 s3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/data_capture/workshop-project-prod/AllTraffic/2022/09/01/02/09-12-543-5b2f079b-ac87-4533-8e89-e431d7

Next, view the content of a single capture file, looking at the first few lines in the captured file.

In [104]:
capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")[-10:-1]
print(capture_file[-1])

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"23,51,1,999,0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.004452417138963938","encoding":"CSV"}},"eventMetadata":{"eventId":"1aa377fb-a3c6-4e54-8d7c-79361eec2d2a","inferenceId":"163","inferenceTime":"2022-09-01T02:15:04Z"},"eventVersion":"0"}


View a single line is present below in a formatted JSON file.

In [105]:
print(json.dumps(json.loads(capture_file[-1]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "23,51,1,999,0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.004452417138963938",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "1aa377fb-a3c6-4e54-8d7c-79361eec2d2a",
    "inferenceId": "163",
    "inferenceTime": "2022-09-01T02:15:04Z"
  },
  "eventVersion": "0"
}


### Generate synthetic traffic

In order to review SageMaker's continuous monitoring capabilities, you will start a thread to generate synthetic traffic to send to the deployed model endpoint. 

The `WorkerThread` class will run continuously on the notebook kernel to generate predictions that are captured and sent to S3 until the kernel is restarted or the thread is explicitly terminated. 

See the cell in the `Cleanup` section to terminate the threads.

If there is no traffic, the monitoring jobs are marked as `Failed` since there is no data to process.

This cell extends a Python Thread class to be able to able to terminate the thread later on without terminating the notebook kernel.

In [16]:
import threading

class WorkerThread(threading.Thread):
    def __init__(self, do_run, *args, **kwargs):
        super(WorkerThread, self).__init__(*args, **kwargs)
        self.__do_run = do_run
        self.__terminate_event = threading.Event()

    def terminate(self):
        self.__terminate_event.set()

    def run(self):
        while not self.__terminate_event.is_set():
            self.__do_run(self.__terminate_event)

Now you define a function that your thread will invoke continuously to send test samples to the model endpoint.

In [17]:
def invoke_endpoint(terminate_event):
    with open("test-samples-no-header.csv", "r") as f:
        i = 0
        for row in f:
            payload = row.rstrip("\n")
            response = sagemaker_runtime_client.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType="text/csv",
                Body=payload,
                InferenceId=str(i),  # unique ID per row
            )
            i += 1
            response["Body"].read()
            time.sleep(1)
            if terminate_event.is_set():
                break


# Keep invoking the endpoint with test data
invoke_endpoint_thread = WorkerThread(do_run=invoke_endpoint)
invoke_endpoint_thread.start()

### Generate synthetic ground truth data

Besides data capture, model bias monitoring execution also requires ground truth data.

In real use cases, ground truth data should be regularly collected and uploaded to designated S3 location. 

The code block below is used to generate fake ground truth data. The first-party merge container will combine captured and ground truth data, and the merged data will be passed to the model bias monitoring job for analysis. Similar to data capture, the model bias monitoring execution will fail if there's no data to merge.

In [18]:
import random

def ground_truth_with_id(inference_id):
    # set random seed to get consistent results.
    random.seed(inference_id) 
    rand = random.random()
    # format required by the merge container.
    return {
        "groundTruthData": {
            # randomly generate positive labels 70% of the time.
            "data": "1" if rand < 0.7 else "0",
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(inference_id),
        },
        "eventVersion": "0",
    }


def upload_ground_truth(upload_time):
    # 180 are the number of rows in data we're sending for inference.
    records = [ground_truth_with_id(i) for i in range(180)]
    fake_records = [json.dumps(r) for r in records]
    data_to_upload = "\n".join(fake_records)
    target_s3_uri = f"{s3_ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(fake_records)} records to", target_s3_uri)
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)

In [19]:
# Generate data for the last hour.
upload_ground_truth(datetime.utcnow() - timedelta(hours=1))

Uploading 180 records to s3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/ground_truth_data/2022-08-30-18-20-30/2022/08/30/18/1435.jsonl


In [20]:
# You can also use the WorkerThread class to continue generating synthetic ground truth data once an hour.
def generate_fake_ground_truth(terminate_event):
    upload_ground_truth(datetime.utcnow())
    for _ in range(0, 60):
        time.sleep(60)
        if terminate_event.is_set():
            break


ground_truth_thread = WorkerThread(do_run=generate_fake_ground_truth)
ground_truth_thread.start()

Uploading 180 records to s3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/ground_truth_data/2022-08-30-18-20-30/2022/08/30/19/1438.jsonl


## Monitor data quality

Data quality monitoring automatically monitors machine learning (ML) models in production and notifies you when data quality issues arise. ML models in production have to make predictions on real-life data that is not carefully curated like most training datasets. If the statistical nature of the data that your model receives while in production drifts away from the nature of the baseline data it was trained on, the model begins to lose accuracy in its predictions. Amazon SageMaker Model Monitor uses rules to detect data drift and alerts you when it happens.

### Configure `DefaultModelMonitor` for detecting data drift.

In [21]:
data_quality_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)



SageMaker Model Monitor provides a built-in container that provides the ability to suggest the constraints automatically for CSV and flat JSON input. This sagemaker-model-monitor-analyzer container also provides you with a range of model monitoring capabilities, including constraint validation against a baseline, and emitting Amazon CloudWatch metrics. This container is based on Spark and is built with [Deequ "unit tests for data"](https://github.com/awslabs/deequ). All column names in your baseline dataset must be compliant with Spark. For column names, use only lowercase characters, and `_` as the only special character.

The training dataset that you used to trained the model is usually a good baseline dataset. The training dataset data schema and the inference dataset schema should exactly match (the number and order of the features). Note that the prediction/output columns are assumed to be the first columns in the training dataset. From the training dataset, you can ask SageMaker to suggest a set of baseline constraints and generate descriptive statistics to explore the data.

### Run data quality baseline job

Start a data quality baseline processing job with `DefaultModelMonitor.suggest_baseline(..)` using the Amazon SageMaker Python SDK. This uses an Amazon SageMaker Model Monitor prebuilt container that generates baseline statistics and suggests baseline constraints for the dataset and writes them to the `output_s3_uri` location that you specify.

The baseline calculations of statistics and constraints are needed as a standard against which data drift and other data quality issues can be detected.

Note: the baseline job will take about 5 min. If the log suppression does not work below, you can clear the cell output with `Edit > Clear Output`.

In [ ]:
data_quality_baseline_job_name = f"DataQualityBaselineJob-{datetime.utcnow():%Y-%m-%d-%H%M}"

data_quality_baseline_job = data_quality_monitor.suggest_baseline(
    job_name=data_quality_baseline_job_name,
    baseline_dataset="train-headers.csv",
    dataset_format=DatasetFormat.csv(header=True),
)

data_quality_baseline_job.wait(logs=False)

Amazon SageMaker Model Monitor prebuilt container computes per column/feature statistics. The statistics are calculated for the baseline dataset and also for the current dataset that is being analyzed. The cell below returns a table of computed baseline feature statistics.

In [23]:
latest_data_quality_baseline_job = data_quality_monitor.latest_baselining_job
schema_df = pd.json_normalize(latest_data_quality_baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,y_yes,Integral,28831,0,0.110749,3193.0,0.313821,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0,..."
1,age,Integral,28831,0,40.020672,1153836.0,10.369838,17.0,95.0,"[{'lower_bound': 17.0, 'upper_bound': 24.8, 'c...",0.64,2048.0,"[[27.0, 46.0, 50.0, 54.0, 46.0, 40.0, 47.0, 51..."
2,campaign,Integral,28831,0,2.574312,74220.0,2.770696,1.0,43.0,"[{'lower_bound': 1.0, 'upper_bound': 5.2, 'cou...",0.64,2048.0,"[[5.0, 1.0, 4.0, 3.0, 2.0, 2.0, 3.0, 2.0, 4.0,..."
3,pdays,Integral,28831,0,963.112102,27767485.0,185.333510,0.0,999.0,"[{'lower_bound': 0.0, 'upper_bound': 99.9, 'co...",0.64,2048.0,"[[999.0, 999.0, 999.0, 999.0, 999.0, 999.0, 99..."
4,previous,Integral,28831,0,0.171101,4933.0,0.493338,0.0,7.0,"[{'lower_bound': 0.0, 'upper_bound': 0.7, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0,..."
5,no_previous_contact,Integral,28831,0,0.963858,27789.0,0.186643,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
6,not_working,Integral,28831,0,0.086331,2489.0,0.280852,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
7,job_admin,Fractional,28831,0,0.252367,7276.0,0.434371,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0,..."
8,job_blue-collar,Fractional,28831,0,0.225001,6487.0,0.417583,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."
9,job_technician,Fractional,28831,0,0.163019,4700.0,0.369383,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


SageMaker Model Monitor will also suggest constraints per feature using the baseline statistics. View them using the cell below:

In [24]:
constraints_df = pd.json_normalize(latest_data_quality_baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

,name,inferred_type,completeness,num_constraints.is_non_negative
0,y_yes,Integral,1.0,True
1,age,Integral,1.0,True
2,campaign,Integral,1.0,True
3,pdays,Integral,1.0,True
4,previous,Integral,1.0,True
5,no_previous_contact,Integral,1.0,True
6,not_working,Integral,1.0,True
7,job_admin,Fractional,1.0,True
8,job_blue-collar,Fractional,1.0,True
9,job_technician,Fractional,1.0,True


### Schedule continuous data quality monitoring

You can create a data monitoring schedule for the endpoint created earlier. 

Use the baseline resources (constraints and statistics) to compare against the real-time traffic hourly.

In [25]:
## Create a data quality monitoring schedule name.
data_quality_monitor_schedule_name = (
    f"xgboost-dm-data-monitoring-schedule-{datetime.utcnow():%Y-%m-%d-%H%M}"
)

In [29]:
# Create an EndpointInput
endpointInput = EndpointInput(
    endpoint_name=predictor.endpoint_name,
    destination="/opt/ml/processing/input_data",
)

In [30]:
# Specify where to write the data quality monitoring results report to.
data_quality_baseline_job_result_uri = f"{s3_baseline_results_path}/data_quality"

response = data_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=data_quality_monitor_schedule_name,
    endpoint_input=endpointInput,
    output_s3_uri=data_quality_baseline_job_result_uri,
    constraints=latest_data_quality_baseline_job.suggested_constraints(),
    # Create the monitoring schedule to execute every hour.    
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

In [31]:
# You will see the monitoring schedule in the 'Scheduled' status
data_quality_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:836443424694:monitoring-schedule/xgboost-dm-data-monitoring-schedule-2022-08-30-1921',
 'MonitoringScheduleName': 'xgboost-dm-data-monitoring-schedule-2022-08-30-1921',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'DataQuality',
 'CreationTime': datetime.datetime(2022, 8, 30, 20, 14, 51, 735000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 8, 30, 20, 15, 3, 259000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'data-quality-job-definition-2022-08-30-20-14-51-032',
  'MonitoringType': 'DataQuality'},
 'EndpointName': 'workshop-project-prod',
 'ResponseMetadata': {'RequestId': '6848cd64-e367-416c-9634-8e1b393855f1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6848cd64-e367-416c-9634-8e1b393855f1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '600',
   'date'

In [32]:
# Check default model monitor created.
predictor.list_monitors()

In [33]:
# Initially there will be no executions since the first execution happens at the top of the hour
# Note that it is common for the execution to launch upto 20 min after the hour.
executions = data_quality_monitor.list_executions()
executions[:5]

No executions found for schedule. monitoring_schedule_name: xgboost-dm-data-monitoring-schedule-2022-08-30-1921


[]

## Monitor model quality

Model quality monitoring jobs monitor the performance of a model by comparing the predictions that the model makes with the actual ground truth labels that the model attempts to predict. To do this, model quality monitoring merges data that is captured from real-time inference with actual labels stored in S3, and then compares the predictions with the actual labels.

### Define `ModelQualityMonitor`

First, define and configure a [`ModelQualityMonitor`](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_monitor.model_monitoring.ModelQualityMonitor) object.

In [34]:
model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=sagemaker_session
)

### Run model quality baseline job

Next, you run a model quality baseline job. As inputs, you need to provide a validation or test dataset with model predictions to establish a model performance baseline. The code block sends the validation dataset against the deployed model endpoint and writes out a CSV file with the labels and predictions in the format `{probability}/{prediction}/{label}` for the `ModelQualityMonitor` to compute a performance baseline. Note you need to provide at least 200 samples to compute model performance metric standard deviations.

The deployed model on your endpoint is an XGBoost binary classifier which outputs predicted probabilities so you will need to score the predictions based on a prediction threshold. By default, it is set as `0.5` below, meaning probabilities above `0.5` will get scored as a `1` and below a `0`.

In [146]:
prediction_threshold = 0.5
model_quality_baseline_job_result_uri = f"{s3_baseline_results_path}/model-quality"
validate_dataset = "validation-with-predictions.csv"

In [147]:
limit = 200  # Need at least 200 samples to compute standard deviations
i = 0
with open(f"{validate_dataset}", "w") as baseline_file:
    baseline_file.write("probability,prediction,label\n")  # our header
    with open("validation.csv", "r") as f:
        for row in f:
            (label, input_cols) = row.split(",", 1)
            probability = float(predictor.predict(input_cols))
            prediction = "1" if probability > prediction_threshold else "0"
            baseline_file.write(f"{probability},{prediction},{label}\n")
            i += 1
            if i > limit:
                break
            print(".", end="", flush=True)
            time.sleep(0.5)
print()
print("Done!")

........................................................................................................................................................................................................
Done!
Uploading 180 records to s3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/ground_truth_data/2022-08-30-18-20-30/2022/09/01/17/1708.jsonl


Call the `suggest_baseline` method of the `ModelQualityMonitor` object to run a baseline job.

Note: this step can take about 8-10 min.

In [37]:
model_quality_baseline_job_name = f"ModelQualityBaselineJob-{datetime.utcnow():%Y-%m-%d-%H%M}"

model_quality_baseline_job = model_quality_monitor.suggest_baseline(
    job_name=model_quality_baseline_job_name,
    baseline_dataset="validation_with_predictions.csv", # The S3 location of the validation dataset.
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri = model_quality_baseline_job_result_uri, # The S3 location to store the results.
    problem_type="BinaryClassification",
    inference_attribute= "prediction", # The column in the dataset that contains predictions.
    probability_attribute= "probability", # The column in the dataset that contains probabilities.
    ground_truth_attribute= "label" # The column in the dataset that contains ground truth labels.
)

model_quality_baseline_job.wait(logs=False)


Job Name:  ModelQualityBaselineJob-2022-08-30-2017
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-836443424694/model-monitor/baselining/ModelQualityBaselineJob-2022-08-30-2017/input/baseline_dataset_input', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/baselines/model-quality', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
.........................................................................................................................................................!

View the suggested model quality baseline constraints.

In [38]:
latest_model_quality_baseline_job = model_quality_monitor.latest_baselining_job
pd.DataFrame(latest_model_quality_baseline_job.suggested_constraints().body_dict["binary_classification_constraints"]).T

,threshold,comparison_operator
recall,0,LessThanThreshold
precision,0,LessThanThreshold
accuracy,0.900498,LessThanThreshold
true_positive_rate,0,LessThanThreshold
true_negative_rate,1,LessThanThreshold
false_positive_rate,0,GreaterThanThreshold
false_negative_rate,1,GreaterThanThreshold
auc,0.393094,LessThanThreshold
f0_5,0,LessThanThreshold
f1,0,LessThanThreshold


### Schedule continuous model quality monitoring

You can create a model monitoring schedule for the endpoint created earlier.

Use the baseline resources (constraints and statistics) to compare against the real-time traffic hourly.

In [39]:
model_quality_monitor_schedule_name = (
    f"xgboost-dm-model-monitoring-schedule-{datetime.utcnow():%Y-%m-%d-%H%M}"
)

In [40]:
# Create an EndpointInput configuration.
endpointInput = EndpointInput(
    endpoint_name=predictor.endpoint_name,
    probability_attribute="0",
    probability_threshold_attribute=0.5,
    destination="/opt/ml/processing/input_data",
)

In [42]:
# Define a monitoring schedule.
response = model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=model_quality_monitor_schedule_name,
    endpoint_input=endpointInput,
    output_s3_uri=model_quality_baseline_job_result_uri,
    problem_type="BinaryClassification",
    ground_truth_input=s3_ground_truth_upload_path,
    constraints=latest_model_quality_baseline_job.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    # enable_cloudwatch_metrics=True,
)

In [43]:
# Check the model monitor was created.
predictor.list_monitors()

In [44]:
# You will see the monitoring schedule in the 'Scheduled' status.
model_quality_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:836443424694:monitoring-schedule/xgboost-dm-model-monitoring-schedule-2022-08-30-2030',
 'MonitoringScheduleName': 'xgboost-dm-model-monitoring-schedule-2022-08-30-2030',
 'MonitoringScheduleStatus': 'Pending',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2022, 8, 30, 21, 9, 40, 177000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 8, 30, 21, 9, 45, 62000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2022-08-30-21-09-39-802',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'workshop-project-prod',
 'ResponseMetadata': {'RequestId': '110bb3ad-6600-4bf3-8ecf-006a3a6df253',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '110bb3ad-6600-4bf3-8ecf-006a3a6df253',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '603',
   'date

In [70]:
# Initially there will be no executions since the first execution happens at the top of the hour
# Note that it is common for the execution to launch up to 20 min after the hour.
executions = model_quality_monitor.list_executions()
executions[:5]

## Monitor model bias

Model bias monitor can detect bias drift of Machine Learning models in a regular basis. Similar to the other monitoring types, the standard procedure of creating a model bias monitor is first baselining and then monitoring schedule.

### Define `ModelBiasMonitor`

First, define a [`ModelBiasMonitor`](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_monitor.clarify_model_monitoring.ModelBiasMonitor) object.

In [46]:
model_bias_monitor = ModelBiasMonitor(
    role=role,
    sagemaker_session=sagemaker_session,
    max_runtime_in_seconds=1800,
)

### Create a model bias baseline job

To execute a model bias baseline job, you need to define a few configurations on the dataset schema, label, and bias definitions in order to establish a baseline to evaluate the model performance against.

#### Configure `DataConfig`

This configuration is defining our dataset schema and label to compute a bias baseline in relation to the label.

In [86]:
train_df.head(1)

,y_yes,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin,job_blue-collar,job_technician,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,0,23,1,999,0,1,0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0


In [106]:
model_bias_baseline_job_result_uri = f"{s3_baseline_results_path}/model_bias"

model_bias_data_config = DataConfig(
    s3_data_input_path="train-headers.csv",
    s3_output_path=model_bias_baseline_job_result_uri,
    label="y_yes",
    headers=train_df.columns.to_list(),
    dataset_type="text/csv",
)

#### Configure `BiasConfig`

Here in the [`BiasConfig`](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html?highlight=clarify#sagemaker.clarify.BiasConfig) you are defining bias by identifying age as a key continuous facet to identify differences in whether certain age groups were proportionally targeted differently by the model.

In [141]:
model_bias_config = BiasConfig(
    label_values_or_threshold=[0.5],
    facet_name="age",
    facet_values_or_threshold=[100],
)

#### Configure `ModelPredictedLabelConfig`

The [`ModelPredictedLabelConfig`](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html?highlight=clarify#sagemaker.clarify.ModelPredictedLabelConfig) specifies how to extract your label from the model output.

In [142]:
model_predicted_label_config = ModelPredictedLabelConfig(
    probability_threshold=0.5,
)

#### Configure `ModelConfig`

The [`ModelConfig`](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html?highlight=clarify#sagemaker.clarify.ModelConfig) provides context to the model baseline job about the model and deploymed model endpoint.

In [143]:
model_config = ModelConfig(
    model_name="Model-gqHK8mt3zYu7",
    instance_count=endpoint_instance_count,
    instance_type=endpoint_instance_type,
    content_type="text/csv",
    accept_type="text/csv",
)

### Run model bias baseline job

Now you can execute the model bias baseline job, providing it the configs for your model, dataset, and bias definitions.

Note: the model bias baseline job can take about 3-5 minutes and depends upon the facet selected to compute the bias metrics.

In [144]:
model_bias_baseline_job = model_bias_monitor.suggest_baseline(
    model_config=model_config,
    data_config=model_bias_data_config,
    bias_config=model_bias_config,
    model_predicted_label_config=model_predicted_label_config,
)

model_bias_baseline_job.wait(logs=False)

print(f"ModelBiasMonitor baselining job: {model_bias_monitor.latest_baselining_job_name}")


Job Name:  baseline-suggestion-job-2022-09-01-16-43-32-050
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-836443424694/baseline-suggestion-job-2022-09-01-16-43-32-050/input/dataset/train-headers.csv', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/baselines/model_bias/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/baselines/model_bias', 'LocalPath': '

UnexpectedStatusException: Error for Processing job baseline-suggestion-job-2022-09-01-16-43-32-050: Failed. Reason: ClientError: No Label values are present in the predicted Label Column,Positive Predicted Index Series contains all False values

View the suggested model bias constraints.

In [ ]:
model_bias_constraints = model_bias_monitor.suggested_constraints()

print(f"ModelBiasMonitor suggested constraints: {model_bias_constraints.file_s3_uri}")

print(S3Downloader.read_file(model_bias_constraints.file_s3_uri))

### Schedule continuous model bias monitoring

Now that you have a baseline, you can call the `create_monitoring_schedule()` method to schedule an hourly monitor to analyze the data with a monitoring schedule. If you have submitted a baselining job, the monitor automatically picks up analysis configuration from the baselining job. If you skip the baselining step or the capture dataset has a different nature from the training dataset, you must provide the analysis configuration.

In [79]:
model_bias_analysis_config = None
if not model_bias_monitor.latest_baselining_job:
    model_bias_analysis_config = BiasAnalysisConfig(
        model_bias_config,
        headers=all_headers,
        label=label_header,
    )

model_bias_monitor.create_monitoring_schedule(
    analysis_config=model_bias_analysis_config,
    output_s3_uri=s3_report_path,
    endpoint_input=EndpointInput(
        endpoint_name=endpoint_name,
        destination="/opt/ml/processing/input/endpoint",
        start_time_offset="-PT1H",
        end_time_offset="-PT0H",
        probability_threshold_attribute=0.5,
    ),
    ground_truth_input=s3_ground_truth_upload_path,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)
print(f"Model bias monitoring schedule: {model_bias_monitor.monitoring_schedule_name}")

Model bias monitoring schedule: monitoring-schedule-2022-09-01-01-48-29-171


In [80]:
# Check the model bias monitor is created.
predictor.list_monitors()

In [ ]:
# You will see the monitoring schedule in the 'Scheduled' status.
model_bias_monitor.describe_schedule()

In [ ]:
# Initially there will be no executions since the first execution happens at the top of the hour
# Note that it is common for the execution to launch up to 20 min after the hour.
executions = model_bias_monitor.list_executions()
executions[:5]

## Monitor feature attribution drift

Model explainability monitoring helps you understand and interpret the predictions made by your ML models. When Model Monitor is configured to monitor model explainability, Amazon SageMaker Clarify automatically detects any drift in relative importance of features and creates reports explaining feature attributions. Monitoring feature attributions can help inform model debugging and error analysis in order to take corrective actions in response to deteriorating model performance such as retraining.  As the model is monitored, customers can view exportable reports and graphs detailing feature attributions in SageMaker Studio and configure alerts in Amazon CloudWatch to receive notifications if it is detected that the attribution values drift beyond a certain threshold. In this section, you will walk through configuring a monitor, creating SHAP feature attribution baselines, and scheduling continuous feature attribution monitoring.

### Define `ModelExplainabilityMonitor`

First, define a [`ModelExplainabilityMonitor`](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_monitor.clarify_model_monitoring.ModelExplainabilityMonitor) object.

In [111]:
model_explainability_monitor = ModelExplainabilityMonitor(
    role=role,
    sagemaker_session=sagemaker_session,
    max_runtime_in_seconds=1800,
)

### Create an explainability baseline job

Just as you have with other monitors so far, you will configure and run a explainability job to suggest feature attribution baselines for use with your monitoring schedule.

#### Define `DataConfig`

Create a [`DataConfig`](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html?highlight=clarify#sagemaker.clarify.DataConfig) object related to configurations of the input and output dataset.

In [120]:
model_explainability_baseline_job_result_uri = f"{s3_baseline_results_path}/model-explainability"

model_explainability_data_config = DataConfig(
    s3_data_input_path="train-headers.csv",
    s3_output_path=model_explainability_baseline_job_result_uri,
    label="y_yes",
    headers=train_df.columns.to_list(),
    dataset_type="text/csv",
)

#### Define Clarify `SHAPConfig` for feature attributions

Here you will define a [SHAPConfig](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html?highlight=clarify#sagemaker.clarify.SHAPConfig) for computing feature attributions. Explanations are typically contrastive, that is, they account for deviations from a baseline. For information on explainability baselines, see [SHAP Baselines for Explainability](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-shap-baseline.html#:~:text=SHAP%20Baselines%20for%20Explainability).

In [126]:
# Here you can compute the feature baselines using the training set, making sure to remove the label.
shap_baseline = [list(train_df.loc[:, train_df.columns != "y_yes"].mean())]

shap_config = SHAPConfig(
    baseline=shap_baseline,
    num_samples=100,
    agg_method="mean_abs",
    save_local_shap_values=False,
)

In [127]:
model_config = ModelConfig(
    model_name="Model-gqHK8mt3zYu7",
    instance_count=endpoint_instance_count,
    instance_type=endpoint_instance_type,
    content_type="text/csv",
    accept_type="text/csv",
)

#### Run explainability baseline job

Now run the explainability baseline job to [suggest feature attribution baselines](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_monitor.clarify_model_monitoring.ModelExplainabilityMonitor.suggest_baseline) using the configurations defined above.

Note: this baseline job can take up to 5-8 min.

In [135]:
model_quality_baseline_job_name = f"ModelExplainabilityBaselineJob-{datetime.utcnow():%Y-%m-%d-%H%M}"

model_explainability_baseline_job = model_explainability_monitor.suggest_baseline(
    job_name=model_quality_baseline_job_name,
    data_config=model_explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
)

model_explainability_baseline_job.wait(logs=False)


Job Name:  ModelExplainabilityBaselineJob-2022-09-01-1543
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-836443424694/ModelExplainabilityBaselineJob-2022-09-01-1543/input/dataset/train-headers.csv', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/baselines/model-explainability/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/baselines/model-explainabilit

View the output model explainability analysis.

In [136]:
model_explainability_constraints = model_explainability_monitor.suggested_constraints()

print(f"ModelExplainabilityMonitor suggested constraints: {model_explainability_constraints.file_s3_uri}")
print(S3Downloader.read_file(model_explainability_constraints.file_s3_uri))

ModelExplainabilityMonitor suggested constraints: s3://sagemaker-us-east-1-836443424694/sagemaker/xgboost-dm-model-monitoring/baselines/model-explainability/analysis.json
{
    "version": "1.0",
    "explanations": {
        "kernel_shap": {
            "label0": {
                "global_shap_values": {
                    "age": 0.00043747580441304984,
                    "campaign": 7.445893741871094e-05,
                    "pdays": 9.922329306070971e-05,
                    "previous": 7.669170123699508e-05,
                    "no_previous_contact": 9.217448527771758e-05,
                    "not_working": 7.352784934097512e-05,
                    "job_admin": 0.0001381603397243405,
                    "job_blue-collar": 8.452694448509678e-05,
                    "job_technician": 0.00010692617184483006,
                    "job_services": 7.459190123434005e-05,
                    "job_management": 7.532721410832243e-05,
                    "job_retired": 7.586593535694722e-05,

### Schedule continuous model explainability monitor

Lastly, you will create a feature attribution [monitoring schedule](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_monitor.clarify_model_monitoring.ModelExplainabilityMonitor.create_monitoring_schedule) to run hourly.

In [137]:
response = model_explainability_monitor.create_monitoring_schedule(
    output_s3_uri=f"{s3_report_path}/model-explainability",
    endpoint_input=endpoint_name,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)

It seems that this object was already used to create an Amazon Model Monitoring Schedule. To create another, first delete the existing one using my_monitor.delete_monitoring_schedule().


ValueError: It seems that this object was already used to create an Amazon Model Monitoring Schedule. To create another, first delete the existing one using my_monitor.delete_monitoring_schedule().

In [138]:
# Check the model explainability monitor is created.
predictor.list_monitors()

In [139]:
# You will see the monitoring schedule in the 'Scheduled' status.
model_explainability_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:836443424694:monitoring-schedule/monitoring-schedule-2022-09-01-03-12-59-633',
 'MonitoringScheduleName': 'monitoring-schedule-2022-09-01-03-12-59-633',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'ModelExplainability',
 'CreationTime': datetime.datetime(2022, 9, 1, 3, 12, 59, 966000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 9, 1, 16, 11, 57, 724000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-explainability-job-definition-2022-09-01-03-12-59-633',
  'MonitoringType': 'ModelExplainability'},
 'EndpointName': 'workshop-project-prod',
 'LastMonitoringExecutionSummary': {'MonitoringScheduleName': 'monitoring-schedule-2022-09-01-03-12-59-633',
  'ScheduledTime': datetime.datetime(2022, 9, 1, 16, 0, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2022, 9, 1, 16, 9, 8, 446000, tzinfo=t

In [140]:
# Initially there will be no executions since the first execution happens at the top of the hour
# Note that it is common for the execution to launch up to 20 min after the hour.
executions = model_explainability_monitor.list_executions()
executions

[]

## Cleanup

Well done! If you are finished with the notebook, run the following cells to terminate lab resources and prevent continued charges.

First, stop the worker threads.

In [ ]:
invoke_endpoint_thread.terminate()
ground_truth_thread.terminate()

Stop and then delete all monitors scheduled to the endpoint.

In [ ]:
model_monitors = predictor.list_monitors()

for monitor in model_monitors:
    monitor.stop_monitoring_schedule()
    monitor.delete_monitoring_schedule()

Finally, delete the endpoint.

In [ ]:
predictor.delete_endpoint()
predictor.delete_model()